In [1]:
# !pip install sentence_transformers
# !pip install git+https://github.com/huggingface/transformers.git

In [1]:
import os
os.chdir("..")
print(os.listdir())
from utils.utils import notebook_line_magic
notebook_line_magic()

['README.md', 'data', '.DS_Store', 'utils', 'notebooks', '.gitignore', '.git']
Line Magic Set


### Data Exploration 
- The nlp dataset has vector of inconsistent lenghts.
- Couple of options (among many):
- Since each integer represents a token. Each vector can be padded with an unused integer to make all vector have a consistent lenght -> then do PCA if needed to reduce dimensions.
- Use a pretrained encoder to generate embeddings.


## data processing
- manually add padding and use sentence transformer to generate embeddings
- use auto tokenize with padding
- manually add padding and then do tf-idf

## modeling ideas
- create a dataloader with a defined bacth size
- use the dataloader to create embeddings and then merge with the UniqueID
- combined embeddings with other features
- build an XGBoost model and a custom torch fully connected network
- you can als get fancy with concatenating the bert model above with a torch classifier 
- knn classifier might also work well with features and embeddings

In [2]:
# add padding to the tokens
import ast
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from utils.data_utils import get_data_df, get_padded_tokens, generate_embeddings_from_tokens

In [5]:
data_df = get_data_df(data_type='val')
generate_embeddings_from_tokens(data_df, embeddings_file_name='data/val/x_val_nlp_embeddings.csv', device="cuda")

Shape of the token features after padding: torch.Size([25000, 2315])


/home/azm0269@auburn.edu/.pyenv/versions/3.10.18/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:236: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


In [30]:
data_path = 'data'
data_type = 'train'
data_path = Path(data_path)
embeddings_file = f'x_{data_type}_nlp_embeddings.csv'
embeddings_file_path = data_path/data_type/embeddings_file

# combine embeddings with other features for modeling
embeddings_df = pd.read_csv(embeddings_file_path).drop(columns=['Unnamed: 0'])
data_df = get_data_df(data_type='train')
data_df = data_df.merge(embeddings_df, on='UniqueID', how='inner')
data_df.set_index('UniqueID').drop(columns=['nlp_feature_vector'])


,Feature1,Feature2,Feature3,Feature4,Target,0,1,2,3,4,...,758,759,760,761,762,763,764,765,766,767
UniqueID,,,,,,,,,,,,,,,,,,,,,
1,3.997380,3.072918,13.681900,-0.012576,1,-1.158507,0.393348,-0.383371,0.312174,-0.704494,...,-0.162445,0.773706,0.699258,-0.170957,-1.031543,0.150399,-0.730741,-1.201290,0.343834,0.282872
2,1.456622,0.306303,15.475434,-1.579090,0,-1.159786,-0.296396,-0.413867,0.648718,-0.279811,...,-0.626879,1.124454,0.414640,0.013904,-1.001874,0.165891,-0.559529,-1.278972,0.566472,0.383504
3,2.648687,-2.123662,15.511687,0.957522,0,-1.610113,0.215146,0.014033,0.488521,-0.197140,...,-0.956866,1.452484,0.456655,-0.090954,-0.821253,0.117969,-0.498769,-1.279072,0.537130,0.465783
4,1.397791,4.013269,16.603704,-0.559142,1,-1.430663,0.371074,-0.264530,0.319842,-0.317677,...,-0.492498,0.697492,0.577387,-0.221709,-0.374156,0.351267,-0.867752,-1.231156,0.192210,0.520025
5,-1.026578,2.810811,13.836552,-2.413276,0,-1.287837,0.399719,0.280019,0.649582,-0.340239,...,-0.670172,1.205005,0.566960,-0.433937,-0.901245,0.369640,-0.483543,-1.358323,0.532113,0.445787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24996,3.860969,2.697122,15.236145,0.545274,1,-1.088969,0.365679,-0.027734,0.403963,-0.052940,...,0.071767,0.663896,0.641669,-0.250575,-0.964048,0.155839,-0.632050,-1.412308,0.353711,0.288682
24997,0.184398,0.903243,14.416246,-1.361478,0,-1.324724,0.123983,-0.157724,0.519878,-0.357197,...,-0.790179,1.030922,0.659210,-0.187817,-0.670324,0.091448,-0.568257,-1.127781,0.489372,0.620648
24998,0.721121,1.898246,14.164095,-3.975487,0,-1.120781,0.126488,0.198821,0.937735,-0.513026,...,-0.022105,0.963705,0.547009,-0.320490,-0.727426,0.172624,-0.633860,-1.451041,0.604985,0.378758


,UniqueID,nlp_feature_vector,Feature1,Feature2,Feature3,Feature4,Target,0,1,2,...,758,759,760,761,762,763,764,765,766,767
0,1,"[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, ...",3.997380,3.072918,13.681900,-0.012576,1,-1.158507,0.393348,-0.383371,...,-0.162445,0.773706,0.699258,-0.170957,-1.031543,0.150399,-0.730741,-1.201290,0.343834,0.282872
1,2,"[1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463,...",1.456622,0.306303,15.475434,-1.579090,0,-1.159786,-0.296396,-0.413867,...,-0.626879,1.124454,0.414640,0.013904,-1.001874,0.165891,-0.559529,-1.278972,0.566472,0.383504
2,3,"[1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 5...",2.648687,-2.123662,15.511687,0.957522,0,-1.610113,0.215146,0.014033,...,-0.956866,1.452484,0.456655,-0.090954,-0.821253,0.117969,-0.498769,-1.279072,0.537130,0.465783
3,4,"[1, 4, 18609, 16085, 33, 2804, 4, 2040, 432, 1...",1.397791,4.013269,16.603704,-0.559142,1,-1.430663,0.371074,-0.264530,...,-0.492498,0.697492,0.577387,-0.221709,-0.374156,0.351267,-0.867752,-1.231156,0.192210,0.520025
4,5,"[1, 249, 1323, 7, 61, 113, 10, 10, 13, 1637, 1...",-1.026578,2.810811,13.836552,-2.413276,0,-1.287837,0.399719,0.280019,...,-0.670172,1.205005,0.566960,-0.433937,-0.901245,0.369640,-0.483543,-1.358323,0.532113,0.445787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,24996,"[1, 14, 9, 6, 2758, 20, 21, 1517, 7, 2078, 5, ...",3.860969,2.697122,15.236145,0.545274,1,-1.088969,0.365679,-0.027734,...,0.071767,0.663896,0.641669,-0.250575,-0.964048,0.155839,-0.632050,-1.412308,0.353711,0.288682
24996,24997,"[1, 4679, 2784, 299, 6, 1042, 37, 80, 81, 233,...",0.184398,0.903243,14.416246,-1.361478,0,-1.324724,0.123983,-0.157724,...,-0.790179,1.030922,0.659210,-0.187817,-0.670324,0.091448,-0.568257,-1.127781,0.489372,0.620648
24997,24998,"[1, 11, 6, 230, 245, 6401, 9, 6, 1225, 446, 2,...",0.721121,1.898246,14.164095,-3.975487,0,-1.120781,0.126488,0.198821,...,-0.022105,0.963705,0.547009,-0.320490,-0.727426,0.172624,-0.633860,-1.451041,0.604985,0.378758
24998,24999,"[1, 1446, 7079, 69, 72, 3305, 13, 610, 930, 8,...",0.432004,0.764494,17.383255,-0.032259,1,-1.139147,0.117001,-0.117284,...,-0.702936,1.204211,0.115389,-0.155379,-0.615290,0.149374,-0.526111,-1.533983,0.720484,0.650374
